<a href="https://colab.research.google.com/github/changyong93/KaggleStruggle/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(9_1%2C_%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D(RNN)).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스로 RNN 구현
- https://wikidocs.net/22886

- RNN 층을 추가하는 코드  
model.add(SimpleRNN(hideen_size)) # 가장 간단한 형태

- 추가 인자를 사용 할 때  
model.add(SimpleRNN(hidden_size, input_shape = (timesteps, input_dim)))

- 다른 표기 #단 M과 N은 정수  
model.add(SimpleRNN(hidden_size, input_length = M, inuput_dim = N))
---
- hidden_size : 은닉 상태의 크기를 정의, 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(Capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128,256, 512, 1024등의 값을 가진다


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
# from keras.models import Sequential #책에선 keras 모듈을 사용
# from keras.layers import SimpleRNN #책에선 keras 모듈을 사용

In [ ]:
model = Sequential()
model.add(SimpleRNN(units = 3, input_shape = (2,10)))
model.summary()

- 출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때, output_dim은 hidden_size의 값인 3
- 이 경우 batch_size를 현 단계에서는 알수 없으므로 (None,3)

- recurrent_weights + input_weights + biases
- resp: (num_features + num_units)* num_units + num_units

- num_units = equals the number of units in the RNN (출력 벡터 차원수)
- num_features = equals the number features of your input(입력 피쳐수)

First you have the recurrent loop, where the state is fed recurrently into the model to generate the next step. Weights for the recurrent step are:

recurrent_weights = num_units*num_units

The secondly you have new input of your sequence at each step.

input_weights = num_features*num_units

(Usually both last RNN state and new input are concatenated and then multiplied with one single weight matrix, nevertheless inputs and last RNN state use different weights)

So now we have the weights, whats missing are the biases - for every unit one bias:

biases = num_units*1

So finally we have the formula:

recurrent_weights + input_weights + biases

or

num_units* num_units + num_features* num_units + biases

=

(num_features + num_units)* num_units + biases

In your cases this means the trainable parameters are:

(3 + 10) * 3 + 3 = 42

In [ ]:
model = Sequential()
model.add(SimpleRNN(units = 3, batch_input_shape = (8,2,10)))
model.summary()

- batch_size를 8로 기재하자, 출력의 크기가 (8, 3)이 된 것을 볼 음
- return_sequences = True로 기재하여 출력 값으로 batch_size, timesteps, output_dim 크기의 3D 텐서를 리턴하도록 모델 생성

In [ ]:
model = Sequential()
model.add(SimpleRNN(units = 3, batch_input_shape = (8,2,10), return_sequences= True))
model.summary()

# 파이썬으로 RNN 구현

In [ ]:
#실제 구현에 앞서 간한단 의사 코드(pseudocode) 작성
# 하기 코드는 의사 코드로 실제 동작하는 코드가 아님

hidden_state_t = 0 #초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: #각 시점마다 입력을 받는다
  output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
  hidden_state_t = output_t # 계산 결과는 현재 시점의 은닉 상태

- 우선 t 시점의 은닉 상태를 hidden_state_t라 변수 선언
- 입력 데이터 길이를 input_length로 선언(이 경우 입력 데이터 길이 = 총 시점(timesteps))
- t 시점의 입력값을 input_t
- 각 메모리 셀은 각 시점마다 input_t와 hidden_state_t(이전 상태의 은닉 상태)를 입력으로 활성화 함수인 하이퍼볼릭 탄젠트 함수를 통해 현 시점의 hidden_state_t 계산산

---
- 이제 실제 RNN층을 동작시키는 코드 구현
- 아래 코드의 이해를 돕기 위해 2D 텐서(timesteps, input_dim)를 입력으로 받았으나, 실제로 케라스에선 3D 텐서(batch_size, timestpes, input_dim)

In [ ]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 무장의 길이
input_dim = 4 # 입력의 차원, NLP에선 보통 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량, 즉 output의 크기 8, 즉 가중치의 개수가 8개

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros(hidden_size,) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬

print(hidden_state_t)

In [ ]:
#가중치 편향 정의
Wx = np.random.random((hidden_size, input_dim)) #(8,4) 크기의 2D 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_size,hidden_size)) #(8,8) 크기의 2D 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_size,)) # (8,) 크기의 1D 텐서 생성 이 값은 편향(bias)

In [ ]:
print(Wx.ndim, Wx.shape)
print(Wh.ndim, Wh.shape)
print(b.ndim, b.shape)

각 텐서 차원, 가중치와 편향의 크기
- Wx : 은닉 상태의 크기 x 입력의 차원
- Wh : 은닉 상태의 크기 x 은닉 상태의 크기
- b : 은닉 상태의 크기

In [ ]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b)
  total_hidden_states.append(list(output_t))
  print(np.shape(total_hidden_states))
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)
#출력 시 값을 깔끔하게 해줌

print(total_hidden_states) #(timesteps, output_dim)의 크기. 이 경우 (10,8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력

# 깊은 순환 신경망(Deep Recurrent Neural Network)
---
- RNN은 다수의 은닉층을 가질 수 있음
- 2개의 깊은(deep) 순환 신경망

In [ ]:
model = Sequential()
model.add(SimpleRNN(units = hidden_size, return_sequences= True))
model.add(SimpleRNN(units = hidden_size, return_sequences= True))

#return_sequence = False로 하거나 별도 기재를 하지 않으면 메모리셀은 하나의 은닉 상태만 출력
# 그리고 이 하나의 값은 마지막 시점(timestep)의 메모리 셀의 은닉 상태

#마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고
#모든 시점의 은닉 상태를 전달하도록 하면 다음 층에 은닉층에 하나 더있는 경우이거나 many to many 문제를 풀 수 있음

# 양방향 순환 신경망(Bidirectional Recurrent Neural Network)
---
- 양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용
-  첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States)를 전달받아 현재의 은닉 상태를 계산

- 두번째 메모리 셀은 뒤 시점의 은닉 상태(Backward States)를 전달 받아 현재의 은닉 상태를 계산

- 이 두 개의 값 모두를 출력층에서 출력값으로 예측하기 위해 사용

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

model = Sequential()
model.add(Bidirectional(SimpleRNN(units = hidden_size, return_sequences=True),
                        input_shape = (timesteps, input_dim)))

In [ ]:
#양방향 다수 은닉층
#단 은닉층을 추가한다고 무조건 성능이 개선되지 않음
#은닉층을 추가하면 학습 양이 많아지지만, 반대로 훈련 데이터도 그만큼 필요

#4개의 은닉층의 경우

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences= True),
                        input_shape = (timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences= True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences= True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences= True)))

# 점검 퀴즈
---
RNN을 제대로 이해했는지 퀴즈를 통해서 확인해보세요! 모델에 대한 설명이 다음과 같을 때, 총 파라미터 개수를 구해보세요.

- Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.
- 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.
- 훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.
- 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.
- 은닉층은 1개입니다.


In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

vocab_size = 5000
embedding_dim = 100
hidden_size = 128

model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = embedding_dim))
model.add(SimpleRNN(units = hidden_size))
model.add(Dense(units = 1, activation = 'sigmoid')) #1개의 은닉층

model.summary()